In [1]:
import gdown 
url = 'https://drive.google.com/uc?id=1gk_2xnpTG9YaOWIsYBgt8fdovtl7LiXA' 
output = 'new-data.csv' 
#gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1gk_2xnpTG9YaOWIsYBgt8fdovtl7LiXA
To: /Users/cgreer/Downloads/new-data.csv
100%|█████████████████████████████████████████████| 288M/288M [01:14<00:00, 3.88MB/s]


'new-data.csv'

In [1]:
import pandas as pd
df = pd.read_csv('/Users/cgreer/Downloads/new-data.csv')

In [2]:
df['date'] = df['date'].astype('datetime64[ns]')

In [3]:
groups = df.groupby(['store','sku'])
keys = []
data = []
for key,item in groups:
    keys.append(key)
    data.append(item)

In [4]:
#%load_ext autotime

In [5]:
from collections import defaultdict
import warnings
import pandas as pd
warnings.filterwarnings("ignore")
import pandas as pd
import plotly
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster 
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [6]:
import ray
import time
#ray.init()

@ray.remote
def grey(df1):
  metadata = MetadataParam(
     time_col="date",  # name of the time column
     value_col="demand",  # name of the value column
     freq="D"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
  )
  forecaster = Forecaster()
  result = forecaster.run_forecast_config(
     df=df1,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=365,  # forecasts 100 steps ahead
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata
      )
  )
  return result

results=[]
for i in data[:5]:
    results.append(grey.remote(i))

#predictions = ray.get(results)

(pid=46022) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/framework/pipeline/pipeline.py:200: UserWarning:
(pid=46022) 
(pid=46022) test_horizon should be <= than 1/3 of the data set size to allow enough data to train a backtest model. Consider reducing to 278. If this is smaller than the forecast_horizon, you will need to make a trade-off between setting test_horizon=forecast_horizon and having enough data left over to properly train a realistic backtest model.
(pid=46022) 
(pid=46028) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/framework/pipeline/pipeline.py:200: UserWarning:
(pid=46028) 
(pid=46028) test_horizon should be <= than 1/3 of the data set size to allow enough data to train a backtest model. Consider reducing to 278. If this is smaller than the forecast_horizon, you will need to make a trade-off between setting test_horizon=forecast_horizon and having enough data left over to properly train a realistic backtest model.

(grey pid=46022) Fitting 1 folds for each of 1 candidates, totalling 1 fits
(grey pid=46028) Fitting 1 folds for each of 1 candidates, totalling 1 fits
(grey pid=46026) Fitting 1 folds for each of 1 candidates, totalling 1 fits
(grey pid=46029) Fitting 1 folds for each of 1 candidates, totalling 1 fits


(pid=46022) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/algo/forecast/silverkite/forecast_simple_silverkite_helper.py:129: UserWarning:
(pid=46022) 
(pid=46022) Requested holiday 'Easter Monday [England, Wales, Northern Ireland]' does not occur in the provided countries
(pid=46022) 
(pid=46028) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/algo/forecast/silverkite/forecast_simple_silverkite_helper.py:129: UserWarning:
(pid=46028) 
(pid=46028) Requested holiday 'Easter Monday [England, Wales, Northern Ireland]' does not occur in the provided countries
(pid=46028) 
(pid=46029) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/algo/forecast/silverkite/forecast_simple_silverkite_helper.py:129: UserWarning:
(pid=46029) 
(pid=46029) Requested holiday 'Easter Monday [England, Wales, Northern Ireland]' does not occur in the provided countries
(pid=46029) 
(pid=46026) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8

(grey pid=46036) Fitting 1 folds for each of 1 candidates, totalling 1 fits


(pid=46036) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/algo/forecast/silverkite/forecast_simple_silverkite_helper.py:129: UserWarning:
(pid=46036) 
(pid=46036) Requested holiday 'Easter Monday [England, Wales, Northern Ireland]' does not occur in the provided countries
(pid=46036) 


In [10]:
ray.get(results[1])

RayTaskError(NotImplementedError): [36mray::grey()[39m (pid=46029, ip=192.168.4.54)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 348, in serialize
    return self._serialize_to_msgpack(value)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 328, in _serialize_to_msgpack
    self._serialize_to_pickle5(metadata, python_objects)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 288, in _serialize_to_pickle5
    raise e
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 284, in _serialize_to_pickle5
    inband = pickle.dumps(
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 580, 

In [8]:
predictions = ray.get(results)

(pid=46022) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/common/evaluation.py:316: UserWarning:
(pid=46022) 
(pid=46022) y_true contains 0. MAPE is undefined.
(pid=46022) 
(pid=46022) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/common/evaluation.py:333: UserWarning:
(pid=46022) 
(pid=46022) y_true contains 0. MedAPE is undefined.
(pid=46022) 
(pid=46022) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:
(pid=46022) 
(pid=46022) Input data has many null values. Missing 30.44% of one input.
(pid=46022) 
(pid=46028) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/common/evaluation.py:316: UserWarning:
(pid=46028) 
(pid=46028) y_true contains 0. MAPE is undefined.
(pid=46028) 
(pid=46028) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/common/evaluation.py:333: UserWarning:
(pid=46028) 
(pid=46028) y_tru

RayTaskError(NotImplementedError): [36mray::grey()[39m (pid=46022, ip=192.168.4.54)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 348, in serialize
    return self._serialize_to_msgpack(value)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 328, in _serialize_to_msgpack
    self._serialize_to_pickle5(metadata, python_objects)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 288, in _serialize_to_pickle5
    raise e
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 284, in _serialize_to_pickle5
    inband = pickle.dumps(
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 580, 

(pid=46036) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/common/evaluation.py:316: UserWarning:
(pid=46036) 
(pid=46036) y_true contains 0. MAPE is undefined.
(pid=46036) 
(pid=46036) /Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/greykite/common/evaluation.py:333: UserWarning:
(pid=46036) 
(pid=46036) y_true contains 0. MedAPE is undefined.
(pid=46036) 
2021-10-25 09:16:34,756	ERROR worker.py:79 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::grey() (pid=46028, ip=192.168.4.54)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 348, in serialize
    return self._serialize_to_msgpack(value)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 328, in _serialize_to_msgpack
    self._serialize_to_pickle5(metadata, python_objects)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", lin

In [11]:
import dill as pickle

In [12]:
predictions = ray.get(results)

RayTaskError(NotImplementedError): [36mray::grey()[39m (pid=46022, ip=192.168.4.54)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 348, in serialize
    return self._serialize_to_msgpack(value)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 328, in _serialize_to_msgpack
    self._serialize_to_pickle5(metadata, python_objects)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 288, in _serialize_to_pickle5
    raise e
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/serialization.py", line 284, in _serialize_to_pickle5
    inband = pickle.dumps(
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/Users/cgreer/miniconda3/envs/koch2/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 580, 